# 01 · Exploración de Datos (EDA) — AVSI
**Artificial Vision Stacking Inspection** · *2025-10-22*
  
Este notebook realiza el **Análisis Exploratorio de Datos** para el proyecto **AVSI**, un sistema de visión por computador para la **inspección del apilamiento de pañales**.
  
**Objetivos del EDA**:
- Verificar estructura de carpetas y conteos de imágenes (dataset de 100 y de 1000).
- Explorar resoluciones, formatos y calidad de imagen.
- Detectar posibles duplicados y outliers simples.
- Revisar distribución de clases (si aplica) y balanceo.
- Preparar insumos para preprocesamiento y entrenamiento.


## 1. Configuración y rutas del proyecto

In [ ]:

from pathlib import Path
import os, sys, math, hashlib, itertools, random, warnings, json
from collections import Counter, defaultdict
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image

# Opciones de visualización
plt.rcParams['figure.dpi'] = 120
pd.set_option('display.max_columns', 50)

# Rutas base
ROOT = Path('.').resolve()
DATA_RAW = ROOT / 'data' / 'raw'
DATA_PROC = ROOT / 'data' / 'processed'

# Datasets esperados (puedes adaptar los nombres si difieren)
DS_SMALL = DATA_RAW / 'dataset_100'
DS_LARGE = DATA_RAW / 'dataset_1000'

# Clases esperadas (si tu dataset está etiquetado por carpetas)
EXPECTED_CLASSES = ['good_stack', 'bad_stack']  # ajustar si aplica

print('ROOT:', ROOT)
print('RAW:', DATA_RAW)
print('SMALL:', DS_SMALL)
print('LARGE:', DS_LARGE)


## 2. Funciones auxiliares

In [ ]:

IMG_EXTS = {'.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff'}

def list_images(folder: Path):
    if not folder.exists():
        return []
    return [p for p in folder.rglob('*') if p.suffix.lower() in IMG_EXTS]

def img_shape(path: Path):
    try:
        img = cv2.imread(str(path))
        if img is None:
            return None
        h, w = img.shape[:2]
        return (h, w)
    except Exception as e:
        return None

def perceptual_hash(path: Path, size=8):
    # Hash simple basado en downscale + diferencia (no cripto)
    try:
        img = Image.open(path).convert('L').resize((size+1, size), Image.LANCZOS)
        diff = []
        for y in range(size):
            for x in range(size):
                diff.append(img.getpixel((x, y)) > img.getpixel((x+1, y)))
        # Convertimos bits a hex
        v = 0
        for i, bit in enumerate(diff):
            v |= (1 if bit else 0) << i
        return hex(v)
    except Exception:
        return None

def brightness_estimate(path: Path):
    try:
        img = cv2.imread(str(path))
        if img is None:
            return None
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        return float(gray.mean())
    except Exception:
        return None

def contrast_estimate(path: Path):
    try:
        img = cv2.imread(str(path))
        if img is None:
            return None
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        return float(gray.std())
    except Exception:
        return None


## 3. Inventario y verificación de datasets

In [ ]:

inv = []
for name, folder in [('dataset_100', DS_SMALL), ('dataset_1000', DS_LARGE)]:
    imgs = list_images(folder)
    inv.append({'dataset': name, 'path': str(folder), 'num_images': len(imgs)})
    
df_inv = pd.DataFrame(inv)
display(df_inv if not df_inv.empty else pd.DataFrame([{'dataset':'(no encontrado)', 'path':str(DS_SMALL), 'num_images':0},
                                                     {'dataset':'(no encontrado)', 'path':str(DS_LARGE), 'num_images':0}]))

print("\nConsejo: si los conteos son 0, verifica las rutas o coloca tus imágenes en data/raw/dataset_100 y data/raw/dataset_1000.")


## 4. Muestreo y métricas básicas (resolución, brillo, contraste)

In [ ]:

def scan_folder(folder: Path, max_samples=2000):
    paths = list_images(folder)
    paths = paths[:max_samples]  # límite razonable
    rows = []
    for p in paths:
        shape = img_shape(p)
        bright = brightness_estimate(p)
        cont = contrast_estimate(p)
        rows.append({
            'path': str(p),
            'fname': p.name,
            'ext': p.suffix.lower(),
            'height': shape[0] if shape else None,
            'width': shape[1] if shape else None,
            'brightness': bright,
            'contrast': cont,
            'phash': perceptual_hash(p)
        })
    return pd.DataFrame(rows)

df_small = scan_folder(DS_SMALL) if DS_SMALL.exists() else pd.DataFrame()
df_large = scan_folder(DS_LARGE) if DS_LARGE.exists() else pd.DataFrame()

print('Muestras dataset_100:', len(df_small))
print('Muestras dataset_1000:', len(df_large))

display(df_small.head())
display(df_large.head())


## 5. Visualizaciones rápidas

In [ ]:

def safe_hist(series, title, xlabel):
    if series is None or series.dropna().empty:
        print(f"[Aviso] No hay datos para {title}")
        return
    plt.figure()
    plt.hist(series.dropna(), bins=30)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel('Frecuencia')
    plt.show()

print('— Dataset 100: tamaños')
if not df_small.empty:
    safe_hist(df_small['width'], 'Ancho (px) — dataset_100', 'width')
    safe_hist(df_small['height'], 'Alto (px) — dataset_100', 'height')
    safe_hist(df_small['brightness'], 'Brillo — dataset_100', 'mean(gray)')
    safe_hist(df_small['contrast'], 'Contraste — dataset_100', 'std(gray)')

print('— Dataset 1000: tamaños')
if not df_large.empty:
    safe_hist(df_large['width'], 'Ancho (px) — dataset_1000', 'width')
    safe_hist(df_large['height'], 'Alto (px) — dataset_1000', 'height')
    safe_hist(df_large['brightness'], 'Brillo — dataset_1000', 'mean(gray)')
    safe_hist(df_large['contrast'], 'Contraste — dataset_1000', 'std(gray)')


## 6. Búsqueda simple de duplicados (perceptual hash)

In [ ]:

def find_duplicates(df):
    if df.empty or 'phash' not in df.columns:
        return pd.DataFrame()
    dup = df.groupby('phash').filter(lambda g: len(g) > 1).sort_values('phash')
    return dup

dup_small = find_duplicates(df_small)
dup_large = find_duplicates(df_large)

print('Duplicados (dataset_100):', len(dup_small))
display(dup_small.head(10))

print('Duplicados (dataset_1000):', len(dup_large))
display(dup_large.head(10))


## 7. Distribución de clases (si el dataset está organizado por carpetas de clase)

In [ ]:

def class_distribution(base_folder: Path, classes: list):
    rows = []
    for cls in classes:
        cpath = base_folder / cls
        rows.append({'class': cls, 'count': len(list_images(cpath))})
    return pd.DataFrame(rows)

if all((DS_SMALL / c).exists() for c in EXPECTED_CLASSES):
    print('— Distribución dataset_100')
    display(class_distribution(DS_SMALL, EXPECTED_CLASSES))

if all((DS_LARGE / c).exists() for c in EXPECTED_CLASSES):
    print('— Distribución dataset_1000')
    display(class_distribution(DS_LARGE, EXPECTED_CLASSES))


## 8. Vista previa de imágenes aleatorias

In [ ]:

def show_samples(df, n=6, title='Muestras'):
    if df.empty:
        print('[Aviso] DataFrame vacío, no hay muestras para mostrar.')
        return
    sample_paths = df.sample(min(n, len(df)), random_state=42)['path'].tolist()
    cols = 3
    rows = math.ceil(len(sample_paths)/cols)
    for i, p in enumerate(sample_paths, 1):
        img = cv2.imread(p)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.figure()
        plt.imshow(img)
        plt.title(Path(p).name)
        plt.axis('off')
        plt.show()

print('— Muestras dataset_100')
show_samples(df_small, n=6, title='dataset_100')

print('— Muestras dataset_1000')
show_samples(df_large, n=6, title='dataset_1000')


## 9. Hallazgos y próximos pasos
**Completar por el equipo (plantilla):**
- **Resoluciones predominantes:** …
- **Rango de brillo/contraste adecuado:** …
- **Duplicados detectados:** …
- **Desbalance de clases:** …
- **Outliers relevantes (borrosas, muy oscuras, etc.):** …

**Acciones para `/02_preprocesamiento.ipynb`:**
- Redimensionar a tamaño estándar (224×224 o el requerido por el backbone).
- Normalización de canales RGB.
- Eliminación o etiquetado de duplicados/outliers.
- Data augmentation (rotaciones, flips, jitter, brillo/contraste).
- División estratificada en train/val/test.


---
### Reproducibilidad
- Estructura esperada:
```
data/
├── raw/
│   ├── dataset_100/
│   └── dataset_1000/
└── processed/
```
- Este notebook **no modifica** `/data/raw`.  
- Los gráficos se generan solo si hay imágenes en las rutas anteriores.
